# 🍊 OPENFOODFACTS - Part 1

<ul><li>Stockage de l'ensemble des données dans une BDD</li>
<li>Connexion à votre base via un script Python</li>
<li>Répondez aux questions SQL en PJ</li></ul>

# 📄 Préparation du data pour import dans base SQL

In [23]:
# import
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sqlalchemy import create_engine
import pandas_profiling

In [24]:
# load dataset
df = pd.read_csv("data/data.csv", sep='\t', low_memory=False)

In [25]:
# shape
df.shape

(1969352, 186)

In [11]:
# sum of null values in the entire dataset
df.isnull().sum().sum()

292300424

In [19]:
# % of missing values in dataset
1969352*186
292300424 / 366299472 * 100

79.79821057454323

In [3]:
# display all columns
pd.set_option('display.max_columns', None)

In [26]:
# new df with selected columns
clean_df = df[['product_name','quantity','nutrition-score-fr_100g','nutriscore_score', 'nutriscore_grade', 'brands', 'origins', 'ingredients_text', 'countries',
            'labels', 'energy-kcal_100g', 'fat_100g', 'saturated-fat_100g', 'fiber_100g', 'carbohydrates_100g', 'sugars_100g', 'sodium_100g', 'additives', 'vitamin-c_100g','nova_group', 'pnns_groups_1', 'pnns_groups_2']]

In [6]:
# keep only columns where 60% or more valid data is available
# clean_df=df.dropna(how='any',axis=1,thresh=df.shape[0]*0.6)

In [6]:
# sum of NaN values on columns
clean_df.isnull().sum().sort_values(ascending=False)

additives                  1969351
origins                    1884353
vitamin-c_100g             1748922
labels                     1549941
quantity                   1480606
nova_group                 1341599
nutriscore_grade           1253813
nutriscore_score           1253813
nutrition-score-fr_100g    1253807
ingredients_text           1249499
brands                      949447
sodium_100g                 494468
energy-kcal_100g            458180
saturated-fat_100g          457544
sugars_100g                 430594
carbohydrates_100g          414965
fat_100g                    414600
product_name                 80982
countries                     6097
pnns_groups_1                 1019
pnns_groups_2                 1017
dtype: int64

In [27]:
# replace NaN by 0 on entire dataframe
clean_df.fillna(0)

,product_name,quantity,nutrition-score-fr_100g,nutriscore_score,nutriscore_grade,brands,origins,ingredients_text,countries,labels,...,saturated-fat_100g,fiber_100g,carbohydrates_100g,sugars_100g,sodium_100g,additives,vitamin-c_100g,nova_group,pnns_groups_1,pnns_groups_2
0,jeunes pousses,0,0.0,0.0,0,endives,0,0,en:france,0,...,0.00,0.0,0.0,0.0,0.000,0,0.0,0.0,unknown,unknown
1,L.casei,0,0.0,0.0,0,0,Spain,"Leche semidesnatada, azucar 6.9% leche desnata...",Spain,0,...,0.90,0.0,9.8,9.8,0.040,0,0.0,0.0,unknown,unknown
2,Vitória crackers,0,0.0,0.0,0,0,0,0,France,0,...,3.08,0.0,70.1,15.0,0.560,0,0.0,0.0,unknown,unknown
3,Cacao,130 g,0.0,0.0,0,0,0,0,France,0,...,0.00,0.0,0.0,0.0,0.000,0,0.0,0.0,unknown,unknown
4,Filetes de pollo empanado,0,0.0,0.0,0,0,0,0,Espagne,0,...,1.00,0.0,0.0,0.0,0.440,0,0.0,0.0,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969347,Sandwich club Rillette poisson combava,0,0.0,0.0,0,0,0,0,en:re,0,...,0.00,0.0,0.0,0.0,0.000,0,0.0,0.0,unknown,unknown
1969348,Thé noir BIO Darjeeling,0,0.0,0.0,0,Pagès,0,0,en:france,0,...,0.00,0.0,0.0,0.0,0.000,0,0.0,0.0,unknown,unknown
1969349,Fati,440 g,0.0,0.0,0,0,0,0,en:be,0,...,0.00,0.0,2.4,0.6,0.256,0,0.0,0.0,unknown,unknown
1969350,Light & Free SKYR A BOIRE,0,0.0,0.0,0,0,0,0,en:france,0,...,0.10,0.0,8.0,7.8,0.060,0,0.0,0.0,unknown,unknown


In [28]:
# replace values "en:fr" , "en:france", "en:FR" by "France" in countries
clean_df['countries'] = clean_df['countries'].replace(['en:fr', 'en:france', 'en:FR', 'fr', 'en:France'],'France')

In [29]:
# keep only the product with countries = France
clean_df = clean_df.loc[clean_df['countries'] == 'France']

In [30]:
clean_df.shape

(760588, 22)

# 🔍 SQL

In [31]:
# database connection
hostname="127.0.0.1"
dbname="openfood"
uname="root"
pwd="pixel"

In [32]:
# create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

In [33]:
# export dataframe to SQL database
clean_df.to_sql('products', engine, index=False)

# ❔ Questions SQL

In [34]:
# connect to MySQL
connection = engine.connect()

<b>Nombre de produits par nutriscore / Quel est le pourcentage de représentation associé?</b>

In [55]:
df_sql_nutriscore = pd.read_sql(
    "SELECT nutriscore_grade, COUNT(product_name) AS 'Count' FROM products GROUP BY nutriscore_grade;", con=connection)
df_sql_nutriscore

,nutriscore_grade,Count
0,None,482821
1,d,81326
2,b,32964
3,a,35099
4,c,56231
5,e,49985


In [54]:
df_sql_nutriscore_perc = pd.read_sql(
    "SELECT nutriscore_grade, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM openfood.products) AS 'Percent' FROM products GROUP by nutriscore_grade;", con=connection)
df_sql_nutriscore_perc

,nutriscore_grade,Percent
0,None,66.33328
1,d,10.71119
2,b,4.34335
3,a,4.62366
4,c,7.40401
5,e,6.58451


<b>Quel est la moyenne des fibres par nutriscore ?</b>

In [53]:
df_sql_fibres = pd.read_sql(
    "SELECT AVG(fiber_100g) AS 'Moyenne fibres', nutriscore_grade FROM openfood.products GROUP BY nutriscore_grade;", con=connection)
df_sql_fibres

,Moyenne fibres,nutriscore_grade
0,3.422850,None
1,2.076879,d
2,2.169106,b
3,4.340926,a
4,2.657559,c
5,1.776429,e


<b>Quels sont le TOP10 des pnns_groups_2 qui comportent le plus d'additif</b>

In [52]:
df_sql_top10 = pd.read_sql(
    "SELECT pnns_groups_2 AS 'PNNS group 2', MAX(additives) AS 'Max additifs' FROM openfood.products GROUP BY pnns_groups_2 LIMIT 10;", con=connection)
df_sql_top10

,PNNS group 2,Max additifs
0,unknown,None
1,Dressings and sauces,None
2,One-dish meals,None
3,Biscuits and cakes,None
4,Fruits,None
5,Meat,None
6,Sweetened beverages,None
7,Cheese,None
8,Bread,None
9,Fish and seafood,None


<b>Afficher le taux de sucre/fat moyen et médian des produits avec un nutriscore A ou B</b>

In [50]:
df_sql_sucre = pd.read_sql(
    "SELECT AVG(sugars_100g) AS 'Moyenne Sucre', product_name AS 'Nom des produits' FROM openfood.products WHERE nutriscore_grade='a' OR nutriscore_grade='b' GROUP BY product_name;", con=connection)
df_sql_sucre

,Moyenne Sucre,Nom des produits
0,4.866667,Salade de carottes râpées
1,20.640000,Compote de poire
2,1.000000,Salade de macedoine de légumes
3,1.500000,Bagel
4,0.600000,BAguette bressan
...,...,...
41702,9.300000,"Jus de fruit Innocent pomme ,peche et poire"
41703,0.000000,Café cappuccino
41704,0.500000,Oignons jaunes 40/60
41705,2.900000,Puree de pomme framboise


<b>Combien avons-nous de produits qui ont un taux de carbohydrates > aux sucres</b>

In [49]:
df_sql_carb = pd.read_sql(
    "SELECT COUNT(DISTINCT product_name) AS 'Total des produits' FROM openfood.products WHERE carbohydrates_100g > sugars_100g;", con=connection)
df_sql_carb

,Total des produits
0,290493
